In [1]:
import numpy as np
import os
import cv2
import imutils
import random
from tqdm import tqdm

**VGG16

In [2]:
import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
# def gray_to_rgb(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     return np.repeat(gray, 3, 2)
# from skimage import color
#, color_mode="grayscale"
# image = color.gray2rgb(grayscale_image)

In [3]:
# trdata = ImageDataGenerator(rescale=1./255, preprocessing_function=gray_to_rgb)
trdata = ImageDataGenerator(rescale=1./255)
traindata = trdata.flow_from_directory(directory=r"C:/Users/Meeta Malviya/Videos/Vgg/train",
                                       target_size=(224,224))
#                                        ,interpolation = 'lanczos:center',
#                                        shuffle = True) 

tsdata = ImageDataGenerator(rescale=1./255)
testdata = tsdata.flow_from_directory(directory=r"C:/Users/Meeta Malviya/Videos/Vgg/test",
                                      target_size=(224,224))
#                                       ,interpolation = 'lanczos:center',
#                                       shuffle = True)

Found 1272 images belonging to 5 classes.
Found 150 images belonging to 5 classes.


In [4]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
for layers in (vggmodel.layers):
#     print(layers)
    layers.trainable = False

In [6]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense,Flatten,Dropout


In [7]:
X = vggmodel.layers[-4].output
X = Dense(460, activation="relu")(X)
X = Dropout(.2)(X)
predictions = Dense(5, activation="softmax")(X)

model_final = Model(input = vggmodel.input, output = predictions)

C:\Users\Meeta Malviya\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [8]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model_final.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [9]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')
model_final.fit_generator(generator= traindata, steps_per_epoch= 2, epochs= 100, validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])
model_final.save_weights("vgg16_1.h5")

Epoch 1/100
2/2 [==============================] - ETA: 26s - loss: 1.9224 - accuracy: 0.125 - 70s 35s/step - loss: 1.8543 - accuracy: 0.0625 - val_loss: 1.5800 - val_accuracy: 0.2188


C:\Users\Meeta Malviya\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
C:\Users\Meeta Malviya\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/100
2/2 [==============================] - ETA: 25s - loss: 1.5480 - accuracy: 0.281 - 75s 37s/step - loss: 1.6304 - accuracy: 0.2656 - val_loss: 1.6316 - val_accuracy: 0.1875
Epoch 3/100
2/2 [==============================] - ETA: 24s - loss: 1.7932 - accuracy: 0.218 - 73s 36s/step - loss: 1.6876 - accuracy: 0.2188 - val_loss: 1.6692 - val_accuracy: 0.0625
Epoch 4/100
2/2 [==============================] - ETA: 24s - loss: 1.7602 - accuracy: 0.125 - 73s 36s/step - loss: 1.8258 - accuracy: 0.1875 - val_loss: 1.6410 - val_accuracy: 0.2188
Epoch 5/100
2/2 [==============================] - ETA: 24s - loss: 1.8771 - accuracy: 0.031 - 65s 33s/step - loss: 1.7606 - accuracy: 0.1406 - val_loss: 1.5956 - val_accuracy: 0.1818
Epoch 6/100
2/2 [==============================] - ETA: 24s - loss: 1.7687 - accuracy: 0.093 - 72s 36s/step - loss: 1.6890 - accuracy: 0.1406 - val_loss: 1.5965 - val_accuracy: 0.1562
Epoch 7/100
2/2 [==============================] - ETA: 23s - loss: 1.7368 - acc

2/2 [==============================] - ETA: 23s - loss: 0.9140 - accuracy: 0.781 - 63s 31s/step - loss: 1.0068 - accuracy: 0.6875 - val_loss: 1.3492 - val_accuracy: 0.4545
Epoch 91/100
2/2 [==============================] - ETA: 23s - loss: 1.0073 - accuracy: 0.656 - 71s 36s/step - loss: 1.0436 - accuracy: 0.6250 - val_loss: 1.2583 - val_accuracy: 0.5625
Epoch 92/100
2/2 [==============================] - ETA: 23s - loss: 1.1162 - accuracy: 0.625 - 71s 36s/step - loss: 1.0520 - accuracy: 0.7188 - val_loss: 1.2135 - val_accuracy: 0.6250
Epoch 93/100
2/2 [==============================] - ETA: 23s - loss: 1.0903 - accuracy: 0.656 - 71s 36s/step - loss: 1.0889 - accuracy: 0.6250 - val_loss: 1.2960 - val_accuracy: 0.5000
Epoch 94/100
2/2 [==============================] - ETA: 23s - loss: 1.0613 - accuracy: 0.750 - 71s 36s/step - loss: 1.0070 - accuracy: 0.7656 - val_loss: 1.2925 - val_accuracy: 0.4688
Epoch 95/100
2/2 [==============================] - ETA: 23s - loss: 0.9627 - accuracy: 

In [10]:
from sklearn.externals import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(model_final, 'VGG16_model.pkl') 

C:\Users\Meeta Malviya\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['VGG16_model_senthil.pkl']

In [ ]:
knn_from_joblib = joblib.load('VGG16_model.pkl')